# 🏁 00. SETUP & OVERVIEW - ETL Pipeline Architecture

## 🎯 MỤC TIÊU:
- Hiểu kiến trúc 3-layer ETL (Raw → Staging → Production)
- Kiểm tra môi trường và kết nối database
- Tổng quan về data flow

## 📚 NỘI DUNG:
1. Kiến trúc ETL 3-layer
2. Kiểm tra kết nối database
3. Tổng quan về dữ liệu
4. Workflow tổng thể

# 🏗️ KIẾN TRÚC ETL 3-LAYER

```
raw_data/          →    raw schema      →    staging schema    →    prod schema
(Parquet files)         (Immutable)          (Cleaned)              (Aggregated)

customers/              raw.customers   →    staging.customers →    prod.customer_metrics
products/               raw.products    →    staging.products
orders/                 raw.orders      →    staging.orders    →    prod.daily_sales
order_items/            raw.order_items →    staging.order_items    prod.monthly_sales
                                                                     prod.daily_category_metrics
                                                                     prod.daily_product_metrics
```

## 📋 CÁC LAYER:

### 1️⃣ **RAW Layer** (Bronze)
- **Mục đích**: Lưu trữ dữ liệu gốc, không thay đổi
- **Đặc điểm**: 
  - Có duplicate, null, lỗi format
  - Có metadata columns (_ingested_at, _source_file, _partition_date)
  - Immutable (không sửa, chỉ append)

### 2️⃣ **STAGING Layer** (Silver)
- **Mục đích**: Dữ liệu đã được làm sạch, chuẩn hóa
- **Đặc điểm**:
  - Remove duplicates
  - Validate data (email format, ranges)
  - Standardize text (capitalize names)
  - Enforce referential integrity

### 3️⃣ **PRODUCTION Layer** (Gold)
- **Mục đích**: Dữ liệu đã được tổng hợp, sẵn sàng cho business
- **Đặc điểm**:
  - Aggregated metrics
  - Denormalized
  - Optimized for reporting

In [1]:
# Import libraries
import sys
sys.path.append('../scripts')

import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

from db_connector import DatabaseConnector
from data_cleaner import DataCleaner
from validators import DataValidator

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## 1. KIỂM TRA KẾT NỐI DATABASE

In [2]:
print("🔌 Kiểm tra kết nối database...")

db = DatabaseConnector()

# Test query
result = db.read_sql("SELECT current_database(), current_user, version()")
print("\n✅ Kết nối thành công!")
print(f"Database: {result['current_database'][0]}")
print(f"User: {result['current_user'][0]}")
print(f"Version: {result['version'][0][:50]}...")

2025-12-20 08:45:35,750 - db_connector - INFO - Database connector initialized for data_engineer@postgres
2025-12-20 08:45:35,803 - db_connector - INFO - Query executed, DataFrame shape: (1, 3)


🔌 Kiểm tra kết nối database...

✅ Kết nối thành công!
Database: data_engineer
User: dataengineer
Version: PostgreSQL 15.15 on x86_64-pc-linux-musl, compiled...


## 2. KIỂM TRA CÁC SCHEMA

In [3]:
print("📂 Kiểm tra schemas...")

schemas_query = """
SELECT schema_name 
FROM information_schema.schemata 
WHERE schema_name IN ('raw', 'staging', 'prod')
ORDER BY schema_name
"""

schemas = db.read_sql(schemas_query)
display(schemas)

2025-12-20 08:45:40,293 - db_connector - INFO - Query executed, DataFrame shape: (3, 1)


📂 Kiểm tra schemas...


,schema_name
0,prod
1,raw
2,staging


## 3. KIỂM TRA TABLES TRONG MỖI SCHEMA

In [4]:
print("📊 Kiểm tra tables...\n")

for schema in ['raw', 'staging', 'prod']:
    try:
        # Get table names
        table_names_query = f"""
        SELECT table_name
        FROM information_schema.tables 
        WHERE table_schema = '{schema}'
        ORDER BY table_name
        """
        tables = db.read_sql(table_names_query)
        
        print(f"{schema.upper()} Schema:")
        print("-" * 50)
        
        for table_name in tables['table_name']:
            count_query = f"SELECT COUNT(*) as count FROM {schema}.{table_name}"
            count = db.read_sql(count_query)['count'][0]
            print(f"  {table_name:.<30} {count:>10,} rows")
        
        print()
            
    except Exception as e:
        print(f"  ⚠️ Error: {e}\n")

2025-12-20 08:45:41,209 - db_connector - INFO - Query executed, DataFrame shape: (4, 1)
2025-12-20 08:45:41,219 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)
2025-12-20 08:45:41,285 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)
2025-12-20 08:45:41,308 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)
2025-12-20 08:45:41,318 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)
2025-12-20 08:45:41,323 - db_connector - INFO - Query executed, DataFrame shape: (4, 1)
2025-12-20 08:45:41,335 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)
2025-12-20 08:45:41,377 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)


📊 Kiểm tra tables...

RAW Schema:
--------------------------------------------------
  customers.....................     11,116 rows
  order_items...................    360,327 rows
  orders........................    119,726 rows
  products......................     36,500 rows

STAGING Schema:
--------------------------------------------------
  customers.....................     10,680 rows
  order_items...................    333,217 rows


2025-12-20 08:45:41,400 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)
2025-12-20 08:45:41,403 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)
2025-12-20 08:45:41,408 - db_connector - INFO - Query executed, DataFrame shape: (5, 1)
2025-12-20 08:45:41,417 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)
2025-12-20 08:45:41,423 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)
2025-12-20 08:45:41,446 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)
2025-12-20 08:45:41,449 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)
2025-12-20 08:45:41,454 - db_connector - INFO - Query executed, DataFrame shape: (1, 1)


  orders........................    110,791 rows
  products......................        100 rows

PROD Schema:
--------------------------------------------------
  customer_metrics..............     10,680 rows
  daily_category_metrics........      2,548 rows
  daily_product_metrics.........     35,837 rows
  daily_sales...................        364 rows
  monthly_sales.................         12 rows



## 4. KIỂM TRA RAW DATA FILES

In [5]:
print("📁 Kiểm tra raw data files...")

raw_data_dir = Path('../raw_data')

if raw_data_dir.exists():
    for entity in ['customers', 'products', 'orders', 'order_items']:
        entity_dir = raw_data_dir / entity
        if entity_dir.exists():
            partitions = sorted([d.name for d in entity_dir.iterdir() if d.is_dir()])
            print(f"\n{entity}:")
            print(f"  Total partitions: {len(partitions)}")
            if partitions:
                print(f"  First: {partitions[0]}")
                print(f"  Last: {partitions[-1]}")
else:
    print("⚠️ Raw data directory not found!")
    print("Run: python scripts/generate_raw_data.py --test-mode")

📁 Kiểm tra raw data files...

customers:
  Total partitions: 365
  First: 2025-01-01
  Last: 2025-12-31

products:
  Total partitions: 365
  First: 2025-01-01
  Last: 2025-12-31

orders:
  Total partitions: 364
  First: 2025-01-02
  Last: 2025-12-31

order_items:
  Total partitions: 364
  First: 2025-01-02
  Last: 2025-12-31


## 5. DATA FLOW VISUALIZATION

In [6]:
print("="*70)
print("📊 DATA FLOW SUMMARY")
print("="*70)

flow_query = """
SELECT 
    'RAW' as layer,
    (SELECT COUNT(*) FROM raw.customers) as customers,
    (SELECT COUNT(*) FROM raw.products) as products,
    (SELECT COUNT(*) FROM raw.orders) as orders,
    (SELECT COUNT(*) FROM raw.order_items) as order_items
UNION ALL
SELECT 
    'STAGING' as layer,
    (SELECT COUNT(*) FROM staging.customers) as customers,
    (SELECT COUNT(*) FROM staging.products) as products,
    (SELECT COUNT(*) FROM staging.orders) as orders,
    (SELECT COUNT(*) FROM staging.order_items) as order_items
UNION ALL
SELECT 
    'PROD' as layer,
    (SELECT COUNT(*) FROM prod.customer_metrics) as customers,
    0 as products,
    (SELECT COUNT(*) FROM prod.daily_sales) as orders,
    0 as order_items
"""

try:
    flow_df = db.read_sql(flow_query)
    display(flow_df)
except Exception as e:
    print(f"⚠️ Some tables might be empty: {e}")

2025-12-20 08:45:54,584 - db_connector - INFO - Query executed, DataFrame shape: (3, 5)


📊 DATA FLOW SUMMARY


,layer,customers,products,orders,order_items
0,RAW,11116,36500,119726,360327
1,STAGING,10680,100,110791,333217
2,PROD,10680,0,364,0


# 🎯 WORKFLOW TỔNG THỂ

## Bước 1: Generate Raw Data
```bash
python scripts/generate_raw_data.py --test-mode
```

## Bước 2: Run ETL Pipeline
```bash
# Option 1: Run từng layer
make etl-run-raw    # Raw layer
make etl-run-stg    # Staging layer
make etl-run-prod   # Production layer

# Option 2: Run full pipeline
make etl-run-full
```

## Bước 3: Explore Data
- Notebook 01: Raw layer exploration
- Notebook 02: Staging transformation
- Notebook 03: Production aggregation

## Bước 4: Validate Data Quality
- Notebook 05: Data quality checks

In [7]:
print("\n✅ Setup complete! Ready to explore ETL pipeline.")
print("\n📚 Next steps:")
print("  1. Run: python scripts/generate_raw_data.py --test-mode")
print("  2. Open: 01_raw_layer_exploration.ipynb")


✅ Setup complete! Ready to explore ETL pipeline.

📚 Next steps:
  1. Run: python scripts/generate_raw_data.py --test-mode
  2. Open: 01_raw_layer_exploration.ipynb
